In [0]:
-- set the catalog and schema
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'main';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT 'fhir_hl7';

SET VARIABLE catalog_use = trim(:catalog_use);
SET VARIABLE schema_use = trim(:schema_use);

USE IDENTIFIER(catalog_use || '.' || schema_use);

-- show catalog and schema 
SELECT current_catalog(), current_schema();

-- show original bronze table
SELECT
  *
FROM
  fhir_bronze
WHERE
  file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
;

-- bronze table with variant data type
SELECT
  *
FROM
  fhir_bronze_variant
WHERE
  file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
;

-- first variant explode to split out file level elements
WITH bronze as (
  FROM fhir_bronze_variant |>
  WHERE file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
)
FROM bronze,
LATERAL variant_explode(fhir);

-- pull out bundle level details 
WITH bronze as (
  FROM fhir_bronze_variant |>
  WHERE file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
)
,bundle_explode as (
  FROM bronze,
  LATERAL variant_explode(fhir)
)
FROM bundle_explode
SELECT * EXCEPT (fhir, pos) |>
PIVOT (
  first(value) as value
  FOR key in ('id', 'entry', 'resourceType', 'type', 'meta', 'timestamp') 
) |>
SELECT *
  ,id::string as bundle_id
  ,resourceType::string as bundle_resourceType
  ,type::string as bundle_type
  ,timestamp::timestamp as bundle_timestamp
  ,meta::struct<lastUpdated: STRING, profile: ARRAY<STRING>, versionId: STRING> as bundle_meta |>
SELECT * EXCEPT (id, resourceType, type, timestamp, meta)
;

-- explode down to resource types
WITH bronze as (
  FROM fhir_bronze_variant |>
  WHERE file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
)
,bundle_explode as (
  FROM bronze,
  LATERAL variant_explode(fhir)
)
,bundle_header as (
  FROM bundle_explode
  SELECT * EXCEPT (fhir, pos) |>
  PIVOT (
    first(value) as value
    FOR key in ('id', 'entry', 'resourceType', 'type', 'meta', 'timestamp') 
  ) |>
  SELECT *
    ,id::string as bundle_id
    ,resourceType::string as bundle_resourceType
    ,type::string as bundle_type
    ,timestamp::timestamp as bundle_timestamp
    ,meta::struct<lastUpdated: STRING, profile: ARRAY<STRING>, versionId: STRING> as bundle_meta |>
  SELECT * EXCEPT (id, resourceType, type, timestamp, meta)
)
FROM bundle_header
,LATERAL variant_explode(entry) as entry_exploded
SELECT * EXCEPT (bundle_header.entry) |>
SELECT *
  ,entry_exploded.value:fullUrl::string as fullUrl
  ,entry_exploded.value:request::struct<method: STRING, url: STRING> as request
  ,entry_exploded.value:resource.resourceType::string as resourceType
  ,entry_exploded.value:resource as resource |>
SELECT * EXCEPT(pos, key, value)
;

-- Measure Report
WITH bronze as (
  FROM fhir_bronze_variant |>
  WHERE file_metadata.file_name IN (
    'RSV_Chester_Friedman_1f368dac-a87d-4462-9698-92406cb66c34.json'
    ,'diabetes-numer-in.json'
  )
)
,bundle_explode as (
  FROM bronze,
  LATERAL variant_explode(fhir)
)
,bundle_header as (
  FROM bundle_explode
  SELECT * EXCEPT (fhir, pos) |>
  PIVOT (
    first(value) as value
    FOR key in ('id', 'entry', 'resourceType', 'type', 'meta', 'timestamp') 
  ) |>
  SELECT *
    ,id::string as bundle_id
    ,resourceType::string as bundle_resourceType
    ,type::string as bundle_type
    ,timestamp::timestamp as bundle_timestamp
    ,meta::struct<lastUpdated: STRING, profile: ARRAY<STRING>, versionId: STRING> as bundle_meta |>
  SELECT * EXCEPT (id, resourceType, type, timestamp, meta)
)
,resources as (
  FROM bundle_header
  ,LATERAL variant_explode(entry) as entry_exploded
  SELECT * EXCEPT (bundle_header.entry) |>
  SELECT *
    ,entry_exploded.value:fullUrl::string as fullUrl
    ,entry_exploded.value:request::struct<method: STRING, url: STRING> as request
    ,entry_exploded.value:resource.resourceType::string as resourceType
    ,entry_exploded.value:resource as resource |>
  SELECT * EXCEPT(pos, key, value)
)
,bunlde_resources as (
  FROM resources |>
  WHERE resourceType = 'Bundle'
)
,measure_report as (
  FROM bunlde_resources
  ,LATERAL variant_explode(resource:entry) as second_pass_entry 
  SELECT *
    ,resourceType as first_pass_resourceType
    ,second_pass_entry.value:resource.resourceType::string as second_pass_resourceType
    ,second_pass_entry.value:resource as second_pass_resource |>
  SELECT * EXCEPT (resource, pos, key, value, resourceType) |>
  WHERE second_pass_resourceType = 'MeasureReport'
)
FROM measure_report
,LATERAL variant_explode(second_pass_resource) as measure_report_resources |>
SELECT * EXCEPT (second_pass_resource, pos) |>
PIVOT (
  first(value) as value
  FOR key in ("contained", "date", "evaluatedResource" ,"extension","group","measure","period","resourceType","status","subject")
) |>
SELECT *
  ,date::timestamp as measure_date
  ,measure::string as measure_str
  ,period:start::timestamp as period_start
  ,period:end::timestamp as period_end
  ,resourceType::string as measure_resourceType
  ,status::string as measure_status 
  ,subject::struct<reference: STRING> as measure_subject |>
SELECT * EXCEPT (period, date, measure, status, subject, resourceType) 
;